<a href="https://colab.research.google.com/github/proyectosRVyderivados/cristina/blob/main/Arbitraje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy-financial

# Arbitraje en el mercado de bonos
Disponemos de dos bonos que maduran a dos años, ambos de cupón explícito y con diferente duración de Macaulay (*duration*).
* El bono A tiene un cupón anual de 10 € y nominal de 1.000 €.
* El bono B tiene un cupón anual de 200 € y nominal de 1.000 €.
* Conocemos la ETTI a un año (5%) y a dos años (20%).
* Supongamos que en el mercado los inversores compran el bono A que es el de mayor TIR y desprecian el bono B. La presión de las compras hace que el precio del bono A suba, haciendo que su TIR baje hasta llegar a coincidir con la del bono B.

Se pide:  
* Calcular el precio de los bonos A y B
* Calcular la TIR de los bonos A y B
* Calcular la *duratin* de ambos bonos
* Ante este mercado desequilibrado:
 - calcular el nuevo precio que tendrá el bono A
 - establecer la estrategia de arbitraje que permitiría obtener ventaja
 - determinar el beneficio obtenido por el arbitraje y en que instante se produciría.



In [7]:
import numpy_financial as npf
import numpy as np
# DATOS
bonoA = [0, 10, 1010]
bonoB = [0, 200, 1200]
etti = [.05, .2]

# PRECIO DE UN BONO CON LA ETTI
def precio_etti(flujos, etti):
    precio = 0
    for i in range(1, len(bonoA)):
        precio += flujos[i] * (1+etti[i-1])**-i
    return precio
# Precios de los Bonos A y B
precioA = precio_etti(bonoA, etti)
precioB = precio_etti(bonoB, etti)
print(f"El precio del bono A es {precioA}")
print(f"El precio del bono B es {precioB}")
# Flujos de caja de los bonos A y B
bonoA[0] = -precioA
bonoB[0] = -precioB
# Calcular la TIR de los bonos
tirA = npf.irr(bonoA)
tirB = npf.irr(bonoB)
print(f"La TIR del bono A es: {tirA:.4%}")
print(f"La TIR del bono B es: {tirB:.4%}")

# Calcular la duración de Macaulay
def duracion(bono):
    tir =npf.irr(bono)
    precio = 0
    numerador = 0
    for i in range(1, len(bonoA)):
        precio += bono[i] * (1+tir)**-i
        numerador += i * bono[i] * (1+tir)**-i
    return numerador / precio

print(f"La duration del bono A es {duracion(bonoA)}")
print(f"La duration del bono B es {duracion(bonoB)}")

# Nuevo precio del bono A en el mercado desequilibrado
precioAprima = npf.npv(tirB, bonoA) + precioA # sumamos el antiguo precio de A ya que va en la posición [0] del array con signo negativo
print(f"El nuevo precio del bono A en el mercado desequilibrado es {precioAprima:.7}")
bonoAprima =[-precioAprima, bonoA[1], bonoA[2]]
print(bonoAprima)

# Bono C = B - 20A'
# bonoC = bonoB - 20 * bonoAprima  # esto da error, necesitamos trabajar con ndarrays
bonoB_arr = np.array(bonoB)
bonoAprima_arr =np.array(bonoAprima)
bonoC_arr = bonoB_arr - 20 * bonoAprima_arr
print(f"El bono C es {bonoC_arr}")
# Ajustamos con un préstamo
prestamo = np.array([bonoC_arr[2] / (1+etti[1])**2, 0, -bonoC_arr[2]])
print(f"El préstamo es {prestamo}")
# Cartera Total
total = bonoC_arr + prestamo
print(f"Cartera resultante del arbitraje {total}")

El precio del bono A es 710.9126984126985
El precio del bono B es 1023.809523809524
La TIR del bono A es: 19.8989%
La TIR del bono B es: 18.4704%
La duration del bono A es 1.988268094271526
La duration del bono B es 1.8351074725250018
El nuevo precio del bono A en el mercado desequilibrado es 728.0583
[-728.0583383461574, 10, 1010]
El bono C es [ 13537.35724311      0.         -19000.        ]
El préstamo es [-13194.44444444      0.          19000.        ]
Cartera resultante del arbitraje [342.91279867   0.           0.        ]
